In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  # workaround for buggy jedi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp core

In [ ]:
#export
import re

# core

> Core functions for SQL formatting

In [ ]:
#hide
from nbdev.showdoc import *

## General helper functions

In [ ]:
#export
def assert_and_print(s_in, s_expected):
    "Assert equality of `s_in` and `s_expected` and print the result of `s_in` if the assertion worked"
    try:
        assert s_in == s_expected
    except:
        print("Assertion failed\n")
        print("Input:\n")
        print(s_in)
        print("\n")
        print("Expected:\n")
        print(s_expected)
        assert s_in == s_expected
    print(s_in)
    return None

In [ ]:
assert_and_print("select asdf\nfrom table1", "select asdf\nfrom table1")

select asdf
from table1


## General formatting

Basic formatting for SQL queries. Let's use an example throughout the core module.

This is how an input could look like

In [ ]:
example_sql = """
create or replace table mytable as -- Mytable example
seLecT a.asdf, b.qwer, -- some comment here
c.asdf, -- some commenT there
case when a.asdf= 1 then 'b' /* here a case comment */
when b.qwer =2 then 'c' else 'd' end as new_field, -- Some comment
b.asdf2 frOm table1 as a leFt join 
table2 as b -- and here a comment
    on a.asdf = b.asdf  /* join this way */
    inner join table3 as c
on a.asdf=c.asdf
whEre a.asdf= 1 -- comment this
anD b.qwer =2 and a.asdf<=1 --comment that
or b.qwer>=5
groUp by a.asdf
"""

and this is how we would like to format it

In [ ]:
expected_sql = """CREATE OR REPLACE TABLE mytable AS -- Mytable example
SELECT a.asdf,
       b.qwer, -- some comment here
       c.asdf, -- some commenT there
       case when a.asdf = 1 then 'b' /* here a case comment */
            when b.qwer = 2 then 'c'
            else 'd' end AS new_field, -- Some comment
       b.asdf2
FROM   table1 AS a
    LEFT JOIN table2 AS b -- and here a comment
        ON a.asdf = b.asdf /* join this way */
    INNER JOIN table3 AS c
        ON a.asdf = c.asdf
WHERE  a.asdf = 1 -- comment this
   and b.qwer = 2
   and a.asdf <= 1 --comment that
    or b.qwer >= 5
GROUP BY a.asdf;"""

Let's start by defining the main statements. The main statements all also require a new line

In [ ]:
#export
MAIN_STATEMENTS = [
    "create table",
    "create or replace table",
    "create view",
    "create or replace view",
    "select",
    "from",
    "left join",
    "inner join",
    "outer join",
    "right join",
    "on",
    "where",
    "group by"
]

We also would like to capitalize the AS operator

In [ ]:
#export
CAPITAL_STATEMENTS = MAIN_STATEMENTS + ["as"]

We need first to capitalize the main statements

In [ ]:
#export
def capitalize_statements(s, statements):
    "Capitalize SQL statements `statements` in string `s`"
    for statement in statements:
        s = re.sub(rf"\b({statement})\b", statement.upper(), s, flags=re.I)
    return s

In [ ]:
print(capitalize_statements(example_sql, CAPITAL_STATEMENTS))


CREATE OR REPLACE TABLE mytable AS -- Mytable example
SELECT a.asdf, b.qwer, -- some comment here
c.asdf, -- some commenT there
case when a.asdf= 1 then 'b' /* here a case comment */
when b.qwer =2 then 'c' else 'd' end AS new_field, -- Some comment
b.asdf2 FROM table1 AS a LEFT JOIN 
table2 AS b -- and here a comment
    ON a.asdf = b.asdf  /* join this way */
    INNER JOIN table3 AS c
ON a.asdf=c.asdf
WHERE a.asdf= 1 -- comment this
anD b.qwer =2 and a.asdf<=1 --comment that
or b.qwer>=5
GROUP BY a.asdf



In [ ]:
assert_and_print(
    capitalize_statements(
        "seLecT asdf, qwer FrOM table1", 
        CAPITAL_STATEMENTS
    ), "SELECT asdf, qwer FROM table1"
)

SELECT asdf, qwer FROM table1


We need also to remove newlines and multiple spaces because the may be arbitrary

In [ ]:
#export
def remove_newlines_mspaces(s):
    """Remove newline and too many spaces characters.
    Newline characters in SELECT statement with comments
    are replaced by special token [EOC] (end of comment)
    because otherwise we cannot format SELECT statements
    with comments properly    
    """
    s = s.strip()  # strip sentence
    s = re.sub(r"[\r\t\f\v ]{2,}", " ", s)  # remove too many whitespaces but not newlines
    s = re.sub(r"\n[\r\t\f\v ]+", r"\n", s)  # remove whitespaces after newline
    s = re.sub(r"[\r\t\f\v ]+\n", r"\n", s)  # remove whitespaces before newline    
    split_s = re.split(r"(select .*?)(from)", s, flags=re.I | re.DOTALL)  # split to get individual select lines
    # add special token [EOC] = End of comment for -- comments
    split_s = [
        re.sub(r"(--.*?)(\n)", r"\1[EOC]", line)  # add special token for end of comment in select lines
        if re.match("select", line, flags=re.I)  # for select statements
        else line  # else no special token
        for line in split_s
    ]
    # add special token [EOC] = End of comment for /*  */ comments
    split_s = [
        re.sub(r"(\/\*.*\*\/)", r"\1[EOC]", line)  # add special token for select lines
        if re.match("select", line, flags=re.I)  # for select statements
        else line  # else no special token
        for line in split_s
    ]
    s = "".join(split_s)  # join all the lines
    s = re.sub("\n", " ", s)  # remove newlines
    s = re.sub(r"\[EOC\][\r\t\f\v ]+", "[EOC]", s)  # remove whitespaces after [EOC]
    s = re.sub(r"[\r\t\f\v ]+\[EOC\]", "[EOC]", s)  # remove whitespaces before [EOC]
    return s

In [ ]:
print(remove_newlines_mspaces(example_sql))

create or replace table mytable as -- Mytable example seLecT a.asdf, b.qwer, -- some comment here[EOC]c.asdf, -- some commenT there[EOC]case when a.asdf= 1 then 'b' /* here a case comment */[EOC]when b.qwer =2 then 'c' else 'd' end as new_field, -- Some comment[EOC]b.asdf2 frOm table1 as a leFt join table2 as b -- and here a comment on a.asdf = b.asdf /* join this way */ inner join table3 as c on a.asdf=c.asdf whEre a.asdf= 1 -- comment this anD b.qwer =2 and a.asdf<=1 --comment that or b.qwer>=5 groUp by a.asdf


In [ ]:
assert_and_print(
    remove_newlines_mspaces(
"""
SELECT asdf, qwer,
 qwer1,    qwer2
FROM table1
"""
    ), "SELECT asdf, qwer, qwer1, qwer2 FROM table1"
)

SELECT asdf, qwer, qwer1, qwer2 FROM table1


With usual comments

In [ ]:
assert_and_print(
    remove_newlines_mspaces("""
SELECT asdf, qwer, -- some comment
 qwer1,    qwer2
FROM table1
WHERE asdf=1
"""), "SELECT asdf, qwer, -- some comment[EOC]qwer1, qwer2 FROM table1 WHERE asdf=1"
)

SELECT asdf, qwer, -- some comment[EOC]qwer1, qwer2 FROM table1 WHERE asdf=1


With other comment form

In [ ]:
assert_and_print(
    remove_newlines_mspaces("""
SELECT asdf, qwer, /*  another comment */
qwer1,    qwer2
FROM table1
WHERE asdf=1
"""), "SELECT asdf, qwer, /* another comment */[EOC]qwer1, qwer2 FROM table1 WHERE asdf=1"
)

SELECT asdf, qwer, /* another comment */[EOC]qwer1, qwer2 FROM table1 WHERE asdf=1


Next we would like to have each main statement in a separate line

In [ ]:
#export
def breakline_statement(s, statements):
    "Write a newline in `s` for all `statements`"
    for statement in statements:
        s = re.sub(rf"\s*({statement})\b", rf"\n{statement}", s, flags=re.I)
    return s

In [ ]:
print(breakline_statement(example_sql, MAIN_STATEMENTS))


create or replace table mytable as -- Mytable example
select a.asdf, b.qwer, -- some comment here
c.asdf, -- some commenT there
case when a.asdf= 1 then 'b' /* here a case comment */
when b.qwer =2 then 'c' else 'd' end as new_field, -- Some comment
b.asdf2
from table1 as a
left join 
table2 as b -- and here a comment
on a.asdf = b.asdf  /* join this way */
inner join table3 as c
on a.asdf=c.asdf
where a.asdf= 1 -- comment this
anD b.qwer =2 and a.asdf<=1 --comment that
or b.qwer>=5
group by a.asdf



In combination with removing newlines and multiple spaces

In [ ]:
print(breakline_statement(remove_newlines_mspaces(example_sql), MAIN_STATEMENTS))


create or replace table mytable as -- Mytable example
select a.asdf, b.qwer, -- some comment here[EOC]c.asdf, -- some commenT there[EOC]case when a.asdf= 1 then 'b' /* here a case comment */[EOC]when b.qwer =2 then 'c' else 'd' end as new_field, -- Some comment[EOC]b.asdf2
from table1 as a
left join table2 as b -- and here a comment
on a.asdf = b.asdf /* join this way */
inner join table3 as c
on a.asdf=c.asdf
where a.asdf= 1 -- comment this anD b.qwer =2 and a.asdf<=1 --comment that or b.qwer>=5
group by a.asdf


We would like to have every non capital statement lower cased but let comments untouched

In [ ]:
#export
def lowercase_query(s):
    "Lowercase query but let comments untouched"
    split_s = s.split("\n")  # split by newline
    split_s = [
        line if re.match(r"^--", line) else # comment at the beginning
        # comment in between
        re.sub(r"(.*)(--.*)$", lambda pat: pat.group(1).lower() + pat.group(2), line)
        if re.match(r".*--", line)
        else line.lower()  # no comment
        for line in split_s
    ]
    s = "\n".join(split_s)
    return s    

In [ ]:
assert_and_print(
    lowercase_query("""
--- My nice view 1 --
Create or Replace VieW view_1 as
seLect asdf, -- Some Comment
qwER,
qwerTy, -- Some other comment
FROM table1
"""),
    """
--- My nice view 1 --
create or replace view view_1 as
select asdf, -- Some Comment
qwer,
qwerty, -- Some other comment
from table1
"""
)


--- My nice view 1 --
create or replace view view_1 as
select asdf, -- Some Comment
qwer,
qwerty, -- Some other comment
from table1



## Specific formatting and validation

Now we will format each statement individually

#### Helper function

In [ ]:
#export
def add_whitespaces_between_symbols(s):
    "Add whitespaces between symbols in line `s`"
    s = re.sub(r"([^\s=!<>])([=!<>]+)", r"\1 \2", s, flags=re.I)  # no space left
    s = re.sub(r"([=!<>]+)([^\s=!<>])", r"\1 \2", s, flags=re.I)  # no space right
    s = re.sub(r"([^\s=!<>])([=!<>]+)([^\s=!<>])", r"\1 \2 \3", s, flags=re.I)  # no space left and right
    return s

In [ ]:
assert_and_print(
    add_whitespaces_between_symbols(
        "WHERE asdf= 1 and qwer=>1 or blabla ='asdf'"
    ), "WHERE asdf = 1 and qwer => 1 or blabla = 'asdf'"
)

WHERE asdf = 1 and qwer => 1 or blabla = 'asdf'


### SELECT

In [ ]:
#export
def format_select(s):
    "Format SELECT statement line `s`"
    # remove [EOC] at end of SELECT
    if re.search(r"\[EOC\]$", s):
        s = re.sub(r"\[EOC\]$", "", s)
    # if comma is found at the end of select statement then remove comma
    if re.search(r"[\w\d]+,\s*$", s, flags=re.I):
        s = re.sub(r"([\w\d]+)(,+)(\s*)$", r"\1", s, flags=re.I)
    elif re.search(r"[\w\d]+,\s*--[\w\d\s]*$", s, flags=re.I):
        s = re.sub(r"([\w\d]+)(,+)(\s*)(--.*)$", r"\1 \4", s, flags=re.I)
    elif re.search(r"[\w\d]+,\s*\/\*.*\*\/$", s, flags=re.I):
        s = re.sub(r"([\w\d]+)(,+)(\s*)(\/\*.*\*\/)$", r"\1 \4", s, flags=re.I)
    s = add_whitespaces_between_symbols(s)  # add whitespaces between symbols
    s = re.sub(r"(,)(\s*)([\w\d]+)", r"\1\n       \3", s)  # add newline after each comma (no comments) and indentation
    s = re.sub(r"\[EOC\]", "\n       ", s)  # replace [EOC] by newline
    s = re.sub(r"(then.*?) ((?:when|else).*?)", r"\1\n\2", s)  # add newline before when or else
    split_s = s.split("\n")
    split_s = [
        line if not re.match("^(?:when|else)", line.strip())
        else " " * 12 + line.strip()
        for line in split_s
    ]
    s = "\n".join(split_s)
    s = s.strip()
    return s

Simple usage without comments

In [ ]:
assert_and_print(
    format_select("select aSdf, cast(qweR as numeric),  Asdf,qwer1"),
    "select aSdf,\n       cast(qweR as numeric),\n       Asdf,\n       qwer1"
)

select aSdf,
       cast(qweR as numeric),
       Asdf,
       qwer1


More advanced usage with comments in SELECT

In [ ]:
assert_and_print(
    format_select("select asdf, cast(qwer as numeric), -- some comment[EOC]ASDF, qwer1"),
    "select asdf,\n       cast(qwer as numeric), -- some comment\n       ASDF,\n       qwer1"
)

select asdf,
       cast(qwer as numeric), -- some comment
       ASDF,
       qwer1


Correcting common mistake on the flow: comma at end of SELECT

In [ ]:
assert_and_print(
    format_select("select qwer1,   asdf,"),
    "select qwer1,\n       asdf"
)

select qwer1,
       asdf


Correcting comma at end of SELECT but having a comment in the last field

In [ ]:
assert_and_print(
    format_select("select qwer1,   asdf, -- this field"),
    "select qwer1,\n       asdf -- this field"
)

select qwer1,
       asdf -- this field


In [ ]:
assert_and_print(
    format_select("select qwer1,   asdf, /* this field */"),
    "select qwer1,\n       asdf /* this field */"
)

select qwer1,
       asdf /* this field */


With `case when` conditions

In [ ]:
assert_and_print(
    format_select("select qwer1, case when abc = 1 then 'a' when abc = 2 then 'b' else 'c' end as qwer2"),
    """
select qwer1,
       case when abc = 1 then 'a'
            when abc = 2 then 'b'
            else 'c' end as qwer2
    """.strip()    
)

select qwer1,
       case when abc = 1 then 'a'
            when abc = 2 then 'b'
            else 'c' end as qwer2


In [ ]:
assert_and_print(
    format_select("select qwer1, case when abc = 1 then 'a' -- first condition[EOC]" +
                  "when abc = 2 then 'b' -- second condition[EOC]" +
                  "else 'c' end as qwer2, /* else condition */[EOC]"
                  "asdf3"
),
    """
select qwer1,
       case when abc = 1 then 'a' -- first condition
            when abc = 2 then 'b' -- second condition
            else 'c' end as qwer2, /* else condition */
       asdf3
    """.strip()    
)

select qwer1,
       case when abc = 1 then 'a' -- first condition
            when abc = 2 then 'b' -- second condition
            else 'c' end as qwer2, /* else condition */
       asdf3


Bad formatted `case when` condition

In [ ]:
assert_and_print(
    format_select("select qwer1, case when abc<= 1 then 'a' -- first condition[EOC]" +
                  "when abc=2 then 'b' -- second condition[EOC]" +
                  "else 'c' end as qwer2, -- else condition[EOC]"
                  "asdf3"
),
    """
select qwer1,
       case when abc <= 1 then 'a' -- first condition
            when abc = 2 then 'b' -- second condition
            else 'c' end as qwer2, -- else condition
       asdf3
    """.strip()    
)

select qwer1,
       case when abc <= 1 then 'a' -- first condition
            when abc = 2 then 'b' -- second condition
            else 'c' end as qwer2, -- else condition
       asdf3


### FROM

In [ ]:
#export
def format_from(s):
    "Format FROM statement line `s`"
    s = re.sub(  # add indentation
        r"(from )(.*)",
        r"\1  \2",
        s,
        flags=re.I
    )
    return s

In [ ]:
assert_and_print(format_from("from table1"), "from   table1")

from   table1


### (LEFT / RIGHT / INNER / OUTER) JOIN

In [ ]:
#export
def format_join(s):
    "Format JOIN statement line `s`"
    s = "    " + s  # add indentation
    return s

In [ ]:
assert_and_print(format_join("inner join table1"), "    inner join table1")

    inner join table1


### ON

In [ ]:
#export
def format_on(s):
    "Format ON statement line `s`"
    s = add_whitespaces_between_symbols(s)  # add whitespaces between symbols in join    
    s = "        " + s  # add indentation
    return s

In [ ]:
assert_and_print(format_on("on a.asdf =b.asdf"), "        on a.asdf = b.asdf")

        on a.asdf = b.asdf


### WHERE

In [ ]:
#export
def format_where(s):
    "Format WHERE statement line `s`"
    s = add_whitespaces_between_symbols(s)  # add whitespaces between symbols
    s = re.sub(r"(where )", r"\1 ", s, flags=re.I)  # add indentation afer WHERE
    s = re.sub(r"\sand", r"\n   and", s, flags=re.I)  # add new line before every 'and' and indentation
    s = re.sub(r"\sor", r"\n    or", s, flags=re.I)  # add new line before every 'or' and indentation    
    return s

In [ ]:
assert_and_print(
    format_where(
        "WHERE asdf= 1 and qwer=1 or blabla ='asdf'"
    ), "WHERE  asdf = 1\n   and qwer = 1\n    or blabla = 'asdf'"
)

WHERE  asdf = 1
   and qwer = 1
    or blabla = 'asdf'


## Format all statements

In [ ]:
#export
def format_statement_line(s):
    "Format statement line `s`"
    statement_funcs = {
        "select": format_select,
        "from": format_from,
        "left join": format_join,
        "right join": format_join,
        "inner join": format_join,
        "outer join": format_join,
        "on": format_on,
        "where": format_where
    }
    for key, format_func in statement_funcs.items():
        if re.match(key, s, flags=re.I):
            s = format_func(s)
    return s

In [ ]:
print(format_statement_line("select asdf, qwer"))

select asdf,
       qwer


In [ ]:
print(format_statement_line("left join table1 as abc"))

    left join table1 as abc


In [ ]:
print(format_statement_line("where asdf=1 and qwer='things' and blabla=0 or stuff=-1"))

where  asdf = 1
   and qwer = 'things'
   and blabla = 0
    or stuff = -1


In [ ]:
#export
def format_statements(s):
    "Format statements lines `s`"
    statement_lines = s.split("\n")
    formatted_lines = [
        format_statement_line(line) for line in statement_lines
    ]
    formatted_s = "\n".join(formatted_lines)
    return formatted_s

In [ ]:
print(format_statements("select asdf, qwer\nfrom table1"))

select asdf,
       qwer
from   table1


### Write a ; at the end of query

In [ ]:
#export
def add_ending_semicolon(s):
    "Add ending semicolon for SQL query `s`"
    s = s.strip()
    if re.match(r".*[^;]$", s, flags=re.DOTALL):
        s = s + ";"
    return s

Basic usage

In [ ]:
assert_and_print(
    add_ending_semicolon("select asdf from table1"), 
    "select asdf from table1;"
)

select asdf from table1;


Also works with multiple lines

In [ ]:
assert_and_print(
    add_ending_semicolon("select asdf\nfrom table1\nwhere asdf = 1"), 
    "select asdf\nfrom table1\nwhere asdf = 1;"
)

select asdf
from table1
where asdf = 1;


It does not add another semicolon if there is already one there

In [ ]:
assert_and_print(
    add_ending_semicolon("select asdf\nfrom table1\nwhere asdf = 1;"), 
    "select asdf\nfrom table1\nwhere asdf = 1;"
)

select asdf
from table1
where asdf = 1;


##  Putting everything together

to format a simple query without subqueries

In [ ]:
#export
def format_simple_sql(s, add_semicolon=True):
    """Format a simple SQL query without subqueries `s`. 
    If `add_semicolon` is True, then add a semicolon at the end
    """
    s = lowercase_query(s)  # everything lowercased but not the comments
    s = remove_newlines_mspaces(s)  # remove newlines and multiple spaces
    s = breakline_statement(s, MAIN_STATEMENTS)  # add breaklines for the main statements
    s = capitalize_statements(s, CAPITAL_STATEMENTS)  # capitalize capital statements
    s = format_statements(s)  # format statements
    if add_semicolon:
        s = add_ending_semicolon(s)  # add ending semicolon if not there yet
    return s

In [ ]:
assert_and_print(
    format_simple_sql(example_sql),
    expected_sql
)

CREATE OR REPLACE TABLE mytable AS -- Mytable example
SELECT a.asdf,
       b.qwer, -- some comment here
       c.asdf, -- some commenT there
       case when a.asdf = 1 then 'b' /* here a case comment */
            when b.qwer = 2 then 'c'
            else 'd' end AS new_field, -- Some comment
       b.asdf2
FROM   table1 AS a
    LEFT JOIN table2 AS b -- and here a comment
        ON a.asdf = b.asdf /* join this way */
    INNER JOIN table3 AS c
        ON a.asdf = c.asdf
WHERE  a.asdf = 1 -- comment this
   and b.qwer = 2
   and a.asdf <= 1 --comment that
    or b.qwer >= 5
GROUP BY a.asdf;


## Queries with subqueries

This is how we could (badly) write a query with subqueries

In [ ]:
example_with_subqueries = """
select asdf, cast(qwer as numeric), -- some comment
qwer1 /* some field */
from 
(select asdf, qwer, /* some nice field */ from table1 where asdf = 1) as a
left 
join (select asdf, qwer2 from table2 where qwer2 = 1) as b
on a.asdf = b.asdf
where qwer1 >= 0
"""
print(example_with_subqueries)


select asdf, cast(qwer as numeric), -- some comment
qwer1 /* some field */
from 
(select asdf, qwer, /* some nice field */ from table1 where asdf = 1) as a
left 
join (select asdf, qwer2 from table2 where qwer2 = 1) as b
on a.asdf = b.asdf
where qwer1 >= 0



and this is the way we would like to have it nicely formatted

In [ ]:
expected_with_subqueries = """
SELECT asdf,
       cast(qwer AS numeric), -- some comment
       qwer1 /* some field */
FROM   (SELECT asdf,
               qwer /* some nice field */
        FROM   table1
        WHERE  asdf = 1) AS a
    LEFT JOIN (SELECT asdf,
                      qwer2
               FROM   table2
               WHERE  qwer2 = 1) AS b
        ON a.asdf = b.asdf
WHERE  qwer1 >= 0;
""".strip()

### Helper functions to handle subqueries

In [ ]:
#export
def extract_outer_subquery(s):
    "Extract outer subquery in query `s`"
    # initialize container for subquery positions
    # in string `s`
    subquery_pos = []
    # auxiliar indicator to get the subquery right
    ind = True
    # counter for parenthesis
    k = 0
    # loop over string characters
    for i, c in enumerate(s):
        if s[i:(i+8)] == "(\nSELECT" and ind: # query start
            subquery_pos.append(i)
            k = 0  # set the parenthesis counter to 0
            # turn off the indicator for the program to know
            # that we already hit the subquery start
            ind = False
        elif c == "(": # if there is a parenthesis not involving a subquery
            k += 1
        elif c == ")" and k == 0 and not ind: # end position for subquery
            subquery_pos.append(i)
            return subquery_pos
        elif c == ")":
            k -= 1

In [ ]:
assert (
    extract_outer_subquery(
        "() () (\nSELECT () (\nSELECT ())) ()"
    ) == [6, 30]
)

In [ ]:
#export
def format_subquery(s, previous_s):
    "Format subquery in line `s` based on indentation on `previous_s`"
    s = re.sub(r"^\(\nSELECT", "(SELECT", s)  # remove newline between parenthesis and SELECT
    # get reference line for the indentation level
    # and remove whitespaces from the left
    ref_line = previous_s.split("\n")[-1].lstrip()
    # if the line contains a JOIN statement then indent with
    # 4 whitespaces
    if re.match(r"\w+ join", ref_line, flags=re.I):
        ref_line = "    " + ref_line
    indentation = len(ref_line) + 1  # get indentation level
    split_s = s.split("\n")  # get lines in subquery
    indented_s = [
        " " * indentation + line  # indent all lines the same
        if not re.match(r"SELECT", line)
        else line
        for line in split_s[1:]
    ]
    # SELECT line + indented lines afterwards
    formatted_split = [split_s[0]] + indented_s
    # concatenate with newline character
    formatted_s = "\n".join(formatted_split)
    return formatted_s

### Main function handling queries with subqueries

In [ ]:
#export
def format_sql(s, add_semicolon=True):
    "Format SQL query with subqueries. If `add_semicolon` is True then add a semicolon at the end"
    s = format_simple_sql(s, add_semicolon)  # basic query formatting
    # get first outer subquery positions
    subquery_pos = extract_outer_subquery(s)
    # loop over subqueries
    while subquery_pos is not None:
        # get split
        split_s = [
            s[0:subquery_pos[0]],
            s[subquery_pos[0]:(subquery_pos[1]+1)],
            s[(subquery_pos[1]+1):]
        ]
        # format subquery (= split_s[1])
        split_s[1] = format_subquery(split_s[1], split_s[0])
        # join main part and subquery
        s = "".join(split_s)
        # get first outer subquery positions
        subquery_pos = extract_outer_subquery(s)
    # remove whitespace between word and parenthesis
    s = re.sub(r"\s\)", ")", s)
    return s

In [ ]:
assert_and_print(
    format_sql(example_with_subqueries),
    expected_with_subqueries
)

SELECT asdf,
       cast(qwer AS numeric), -- some comment
       qwer1 /* some field */
FROM   (SELECT asdf,
               qwer /* some nice field */
        FROM   table1
        WHERE  asdf = 1) AS a
    LEFT JOIN (SELECT asdf,
                      qwer2
               FROM   table2
               WHERE  qwer2 = 1) AS b
        ON a.asdf = b.asdf
WHERE  qwer1 >= 0;


It even works with simple queries without subqueries, therefore generalizing the `format_simple_sql()` function

In [ ]:
assert_and_print(
    format_sql(example_sql),
    expected_sql
)

CREATE OR REPLACE TABLE mytable AS -- Mytable example
SELECT a.asdf,
       b.qwer, -- some comment here
       c.asdf, -- some commenT there
       case when a.asdf = 1 then 'b' /* here a case comment */
            when b.qwer = 2 then 'c'
            else 'd' end AS new_field, -- Some comment
       b.asdf2
FROM   table1 AS a
    LEFT JOIN table2 AS b -- and here a comment
        ON a.asdf = b.asdf /* join this way */
    INNER JOIN table3 AS c
        ON a.asdf = c.asdf
WHERE  a.asdf = 1 -- comment this
   and b.qwer = 2
   and a.asdf <= 1 --comment that
    or b.qwer >= 5
GROUP BY a.asdf;


## Nested subqueries

The function is also robust against nested subqueries

In [ ]:
example_nested_subqueries = """
select asdf, qwer
from (select a.asdf
    from (select asdf, qwer from table2) as a
        inner join (select asdf, qwer from table3) as b
            on a.qwer = b.qwer
)
"""

In [ ]:
expected_nested = """SELECT asdf,
       qwer
FROM   (SELECT a.asdf
        FROM   (SELECT asdf,
                       qwer
                FROM   table2) AS a
            INNER JOIN (SELECT asdf,
                               qwer
                        FROM   table3) AS b
                ON a.qwer = b.qwer);"""

In [ ]:
assert_and_print(
    format_sql(example_nested_subqueries),
    expected_nested
)

SELECT asdf,
       qwer
FROM   (SELECT a.asdf
        FROM   (SELECT asdf,
                       qwer
                FROM   table2) AS a
            INNER JOIN (SELECT asdf,
                               qwer
                        FROM   table3) AS b
                ON a.qwer = b.qwer);


### More convoluted nested subquery

In [ ]:
example_convoluted = """
select asdf
from (
    select asdf, qwer, /* some comment */
    from (select a.asdf, b.qwer, --some comment
          from (select asdf 
                from table1) as a 
            right join (select qwer 
                        from table2) as b
                on a.asdf = b.asdf)
)
"""

In [ ]:
expected_convoluted = """SELECT asdf
FROM   (SELECT asdf,
               qwer /* some comment */
        FROM   (SELECT a.asdf,
                       b.qwer --some comment
                FROM   (SELECT asdf
                        FROM   table1) AS a
                    RIGHT JOIN (SELECT qwer
                                FROM   table2) AS b
                        ON a.asdf = b.asdf));"""

In [ ]:
assert_and_print(
    format_sql(example_convoluted),
    expected_convoluted
)

SELECT asdf
FROM   (SELECT asdf,
               qwer /* some comment */
        FROM   (SELECT a.asdf,
                       b.qwer --some comment
                FROM   (SELECT asdf
                        FROM   table1) AS a
                    RIGHT JOIN (SELECT qwer
                                FROM   table2) AS b
                        ON a.asdf = b.asdf));


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_format_file.ipynb.
Converted index.ipynb.
